In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.colors as mcolors

In [ ]:
# Adjust these for your organism
mu = 4.6e-9
gen = 2

In [ ]:
def transform_data(file):
    data = pd.read_csv(file, sep='\t')
    data['generation'] = data['left_time_boundary'] / mu * gen
    data['PopSize'] = (1 / data['lambda']) / mu
    data['MSMSTime'] = data['generation'] / 100000 / 4
    data['MSMSSize'] = data['PopSize'] / 100000
    
    return data

def get_populations(file):
    populations = {}
    with open(file) as f:
        for line in f:
            line = line.strip().split()
            populations[line[0]] = line[1:]
    
    return list(populations.keys())

def cross_population_files(populations):
    if len(populations) == 1:
        raise ValueError("At least two populations are required for cross-population analysis.")

    combined_files = {}

    for i, pop1 in enumerate(populations[:-1]):
        for j, pop2 in enumerate(populations[i+1:]):
            combined_files[(pop1, pop2)] = (f"../results/cross/{pop1}-{pop2}.combined.msmc2")

    return combined_files

def transform_cross_data(file):
    data = pd.read_csv(file, sep='\t')
    data['generation'] = data['left_time_boundary'] / mu * gen
    data['cross_rate'] = 2 * data['lambda_01'] / (data['lambda_00'] + data['lambda_11'])
    
    return data

In [ ]:
populations = get_populations('../resources/populations.txt')
cross_populations = cross_population_files(populations)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 10))

for pop in populations:
    file_path = f'../results/{pop}/{pop}.msmc2.final.txt'
    if os.path.exists(file_path):
        data = transform_data(file_path)
    
        ax.step(data['generation'], data['PopSize'], label=pop)
        
ax.set_xscale('log')
ax.set_yscale('log')
ax.set_xlabel('years')
ax.set_ylabel('Effective population size')
        
ax.legend()

In [ ]:
# The default just has one population group per population
population_groups = {pop: [pop] for pop in populations}

# But it is possible to define more complex groups, where a group contains multiple populations, which
# will be colored the same in the plot. Here is an example configuration from our European robin dataset:
# population_groups = {
#     'mig': ['Spain_N', 'Spain_SD', 'Spain_GU', 'Wold'],
#     'res_cont': ['S_Spain', 'Ceuta', 'Morocco'],
#     'res_island': ['Terceira', 'Sao_Miguel', 'Canary', 'Madeira']
# }

# The default just selects colors from the Tableau color palette.
color_pallete = list(mcolors.TABLEAU_COLORS.values())
colors = {}
color_index = 0
for pop in population_groups:
    if len(population_groups[pop]) > 1:
        colors[pop] = color_pallete[color_index % len(color_pallete)]
        color_index += 1

for cross_pop in cross_populations:
    colors[cross_pop] = color_pallete[color_index % len(color_pallete)]
    color_index += 1

# You may also define your own colors by creating a dictionary that contains both single population groups
# with more than one population, as well as tuples of two population groups for their cross-coalescence. 
# Here is an example configuration from our European robin dataset:
# colors = {
#     'mig': 'grey',
#     'res_cont': 'red',
#     'res_island': 'green',
#     ('mig', 'res_cont'): 'black',
#     ('mig', 'res_island'): 'gold',
#     ('res_cont', 'res_island'): 'purple'
# }


In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 10))

rev_pop_groups = {v: k for k, vs in population_groups.items() for v in vs}

for (pop1, pop2), file in cross_populations.items():
    if pop1 not in rev_pop_groups or pop2 not in rev_pop_groups:
        continue
    group1 = rev_pop_groups[pop1]
    group2 = rev_pop_groups[pop2]
    
    if group1 == group2:
        group = group1
    else:
        group = (group1, group2)
        if group not in colors:
            group = (group2, group1)
    
    if os.path.exists(file):
        data = transform_cross_data(file)
        
        ax.step(data['generation'], data['cross_rate'], color=colors[group])

for group, color in colors.items():
    if isinstance(group, tuple):
        label = f"{group[0]} x {group[1]}"
    else:
        label = group
    
    ax.plot([], [], color=color, label=label)    

ax.set_ylim(0, 1)

ax.set_xscale('log')

ax.set_xlabel('years')
ax.set_ylabel('Cross-coalescence rate')

ax.legend()